## Loading useful libraries and modules

In [1]:
import re
import string

import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import TweetTokenizer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from pprint import pprint
from tqdm.notebook import tqdm

## Loading training and testing data

In [2]:
# Load train data
usecols = ['id', 'review_title', 'review_text']
raw_train_data = pd.read_csv('../input/i2a2-nlp-2021-sentiment-analysis/train.csv', index_col='id', usecols=usecols + ['rating'])

# Load test data
raw_test_data = pd.read_csv('../input/i2a2-nlp-2021-sentiment-analysis/train.csv', index_col='id', usecols=usecols)

In [3]:
# Concatenate text features
raw_train_data['review'] = raw_train_data['review_title'] + ' ' + raw_train_data['review_text']
raw_test_data['review'] = raw_test_data['review_title'] + ' ' + raw_test_data['review_text']

with pd.option_context('display.max_colwidth', None):
    display(raw_train_data[['review', 'rating']].head())

,review,rating
id,,
0,Americanas Nao tem compromisso Comprei um TV Box dia 14 /12/ 2017 e ate agora as americanas não Devovel o meu dinheiro. Ja Fiz de tudo pre entrar em contato com essa empresa mas os fones que sao disponiveis por ela nao existem. pre mim isso se chama CALOTE.,1
1,"muito bom produto Considerando seu custo, é ótimo som com muita qualidade.",4
2,"Excelente! Simplesmente é a melhor que já vi... ela é linda, grande, fácil de montar e praticamente ela se autoconfigura sozinha. Parece que vc está no computador. O som é ótimo e a definição de imagem me deixou apaixonada. A entrega foi muito rápida e gostei muito da presteza e agilidade da transportadora Plimor. Pude acompanhar cada passo da entrega. Parabéns!",5
3,Entrega antes do prazo! Estou muito satisfeita com o produto é a entrega no prazo.,4
4,"Muito bonito. A gaiola é muito bonita. Colorida e grande. Algumas grades ficaram meio soltas,mas ainda valeu a pena.",3


## Exploratory data analysis

In [23]:
corpus = ' '.join([*raw_train_data['review']])
vocab = nltk.FreqDist(TweetTokenizer().tokenize(corpus))

In [26]:
print(f'The training data has {len(vocab):,} unique words.', '---' , sep='\n', end='\n\n')

# Show the 100 most common words
pprint(vocab.most_common(100),  compact=True)

The training data has 69,407 unique words.
---

[('.', 139325), (',', 134038), ('e', 75133), ('o', 68277), ('de', 65344),
 ('!', 58789), ('a', 56868), ('produto', 56634), ('que', 46407), ('não', 40569),
 ('muito', 38656), ('do', 35955), ('é', 32916), ('com', 30187), ('bom', 24561),
 ('para', 23500), ('um', 22703), ('Produto', 17562), ('da', 17371),
 ('O', 16326), ('em', 16077), ('no', 15774), ('entrega', 14969), ('na', 13602),
 ('qualidade', 13323), ('mais', 12977), ('...', 12698), ('uma', 12671),
 ('mas', 12405), ('Não', 12036), ('bem', 12021), ('recomendo', 12004),
 ('prazo', 11466), ('Muito', 10800), ('Gostei', 10595), ('as', 10429),
 ('foi', 10361), ('A', 10037), ('chegou', 9429), ('excelente', 9063),
 ('antes', 8826), ('eu', 8534), ('se', 8329), ('tem', 8327), ('recebi', 8170),
 ('Excelente', 7937), ('Ótimo', 7638), ('por', 7581), ('meu', 7562),
 ('como', 7431), ('Recomendo', 7249), ('me', 6988), ('compra', 6808),
 ('minha', 6719), ('veio', 6657), ('boa', 6556), ('ótimo', 6550), (

In [6]:
# Show the 100 least common words
pprint(vocab.most_common()[:-101:-1],  compact=True)

[('MONTAGEM.ME', 1), ('parebens', 1), ('reclamar.Pra', 1), ('retificar', 1),
 ('totalizar', 1), ('contabilizando', 1), ('encurvados', 1), ('Vergonhosa', 1),
 ('Surprendente', 1), ('locar', 1), ('morava', 1), ('finda', 1),
 ('Desamassa', 1), ('AJUDE', 1), ('escovo', 1), ('pesquisávamos', 1),
 ('HX318C10FR', 1), ('CL10', 1), ('1866Mhz', 1), ('FURY', 1), ('MU6120', 1),
 ('ort', 1), ('8-p', 1), ('todo.um', 1), ('17,18', 1), ('traições', 1),
 ('dramas', 1), ('korai', 1), ('Mastercad', 1), ('credores', 1), ('sugunda', 1),
 ('CERCA', 1), ('performa', 1), ('excelente.Consistente', 1), ('ahshsh', 1),
 ('Caçadores', 1), ('Veda', 1), ('descepicionado', 1), ('recebi.Estou', 1),
 ('EDN', 1), ('FECHA', 1), ('327', 1), ('ELM', 1), ('TELEFONEMAS', 1),
 ('Inconformada', 1), ('cente', 1), ('fixá-la', 1), ('sgestão', 1),
 ('Inutilizando', 1), ('agor', 1), ('inrresponssabilidade', 1),
 ('Descepção', 1), ('flashes', 1), ('Mamiss', 1), ('cumpádis', 1),
 ('casebre', 1), ('lampiões', 1), ('pavios', 1), ('gera

In [7]:
long_words = [word for word in vocab if len(word) > 20]

# Show some long words
pprint(long_words[:10], compact=True)

['https://www.americanas.com.br/',
 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx', 'LARANJAS.ASSIST.TECNICA',
 'Uhuhuhuhuhuhuhihuuhuuhuhuhuuhuhuhuh',
 'hjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjj',
 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx',
 'tooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooop',
 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx',
 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx',
 'HUAHAUHUHAUAHUAHUAHUAHUAHAUHAUAHUAHUAHUAHUAHAUHAUHAUHAUHAU']


In [8]:
short_words = [word for word in vocab if len(word) <= 3]

# Show some short words
pprint(short_words[:110], compact=True)

['Nao', 'tem', 'um', 'TV', 'Box', 'dia', '14', '/', '12', 'e', 'ate', 'as',
 'não', 'o', 'meu', '.', 'Ja', 'Fiz', 'de', 'pre', 'em', 'com', 'mas', 'os',
 'que', 'sao', 'por', 'ela', 'nao', 'mim', 'se', 'bom', 'seu', ',', 'é', 'som',
 '!', 'a', 'já', 'vi', '...', 'vc', 'no', 'O', 'me', 'A', 'foi', 'da', 'do',
 'hp', 'sem', '..', 'Boa', 'ele', 'nem', 'bem', 'boa', 'Com', 'eu', 'pra', 'q',
 'só', 'voz', '10', 'fiz', 'As', 'na', 'Vc', 'qdo', 'vê', 'Ela', 'Mas', 'tá',
 'Tô', 'ano', 'uso', 'J5', 'Não', 'ao', 'É', 'mão', 'era', 'eté', '22', '06',
 'Já', 'ser', '(', ')', 'Eu', 'sou', 'até', 'hj', 'n', 'deu', 'E', 'uma', 'há',
 'ou', 'vou', 'B', 'aos', 'sai', 'dor', 'fiu', 'BOM', 'JÁ', 'UM', 'Só', 'ter']


## Preprocessing raw text data

In [9]:
# Compile regular expressions
remove_url = re.compile(r'https?://\S+|www\.\S+')
remove_email = re.compile(r'\S+@\S+')
remove_duplicate_word = re.compile(r'\b(\w+?)\1+')
remove_duplicate_char = re.compile(r'([^rs])(?=\1+)|(r)(?=r{2,})|(s)(?=s{2,})')
remove_number = re.compile(r'\d+')
remove_extra_space = re.compile(r'\s+')

# Load punctuation
punctuation = [*string.punctuation]
punctuation.extend(['º', 'ª'])

def preprocess_text(text: str) -> str:

    # Convert to lowercase
    text = text.lower() 
    
    # Apply regular expressions
    text = remove_url.sub('', text)    # remove urls
    text = remove_email.sub('', text)  # remove emails
    text = remove_duplicate_word.sub(r'\1', text) # remove duplicate words
    text = remove_duplicate_char.sub('', text)    # remove duplicate chars; except "rr" and "ss" digraphs
    text = remove_number.sub(' ', text)  # remove numbers

    ## Expand abbreviatons
    text = re.sub(r'\b(n|ñ)([aãâ]o)?\b', ' não ', text)
    text = re.sub(r'\bt[aá]\b', ' está ', text)
    text = re.sub(r'\b(p(r[oaá])?)\b', ' para ', text)
    text = re.sub(r'\bq\b', ' que ', text)
    text = re.sub(r'\bpq[s]?\b', ' porque ', text)
    text = re.sub(r'\btb[m|n]?\b', ' também ', text)
    text = re.sub(r'\vc[s]?\b', ' você ', text)
    text = re.sub(r'\bmt[ao]?s?\b', ' muito ', text)
    text = re.sub(r'\b(p(r[oaá]s?)?)\b', ' para ', text)
    text = re.sub(r'\bhj\b', ' hoje ', text)
    text = re.sub(r'\bobs\b', ' observação ', text)
    text = re.sub(r'\beh\b', ' é ', text)

    # Preprocess long words
    text = re.sub(r'(ótimo)[v]?\1+', r' \1 ', text)
    text = re.sub(r'\b(ok)\1+', r' \1 ', text)
    text = re.sub(r'\b(natural)', r' \1 ', text)
    text = re.sub(r'(((bla)(ba)?)+\2)', r' \2 ', text)
    text = re.sub(r'(altura|largura)x', r' \1 ', text)
    text = re.sub(r'(compra|embalagem)x', r' \1 ', text) 
    text = re.sub(r'(ruim|regular|bom|[oó]timo|excelente)', r' \1 ', text)

    # Remove trailing spaces
    text = remove_extra_space.sub(' ', text)

    # Instatiate a TweetTokenizer object
    tokenizer = TweetTokenizer(preserve_case=False, # lowercasing
                               reduce_len=True, 
                               strip_handles=True)  # remove mentions
                               
    # Tokenize the text    
    tokens = tokenizer.tokenize(text)

    # Remove punctuation
    tokens = [token for token in tokens if token not in punctuation]
    
    # Remove non-alphabetic and longer than twenty chars words
    tokens = [token for token in tokens if token.isalpha() and len(token) <= 20]

    return ' '.join(tokens)

In [10]:
%%time
raw_train_data['review_clean'] = raw_train_data['review'].apply(preprocess_text)
raw_test_data['review_clean'] = raw_test_data['review'].apply(preprocess_text)

CPU times: user 1min 57s, sys: 11.3 ms, total: 1min 57s
Wall time: 1min 57s


## After data preprocessing

In [21]:
corpus = ' '.join([*raw_train_data['review_clean']])
vocab = nltk.FreqDist(TweetTokenizer().tokenize(corpus))

In [22]:
print(f'The training data has {len(vocab):,} unique words.', '---' , sep='\n', end='\n\n')

# Show the 100 most common words
pprint(vocab.most_common(100),  compact=True)

The training data has 42,530 unique words.
---

[('o', 84646), ('e', 80601), ('produto', 77809), ('de', 68492), ('a', 67199),
 ('não', 63056), ('muito', 52182), ('que', 51063), ('do', 37285), ('é', 36762),
 ('para', 33746), ('com', 31999), ('bom', 31151), ('um', 25072),
 ('recomendo', 20083), ('entrega', 19041), ('da', 18168), ('excelente', 18066),
 ('no', 17381), ('em', 17267), ('gostei', 17102), ('ótimo', 14892),
 ('na', 14806), ('mas', 14620), ('qualidade', 14587), ('uma', 14137),
 ('mais', 13991), ('bem', 12748), ('chegou', 12670), ('prazo', 12031),
 ('as', 11823), ('foi', 11501), ('eu', 11420), ('recebi', 10218), ('se', 9731),
 ('comprei', 9434), ('tem', 9313), ('antes', 9174), ('como', 9034),
 ('meu', 8975), ('por', 8676), ('americanas', 8496), ('super', 8452),
 ('me', 8109), ('boa', 8109), ('já', 8051), ('minha', 7979), ('veio', 7751),
 ('ainda', 7650), ('estou', 7452), ('compra', 7448), ('sem', 7232),
 ('os', 7143), ('até', 7133), ('dia', 6594), ('só', 6300), ('pois', 5853),
 (

In [13]:
# Show the 100 least common words
pprint(vocab.most_common()[:-101:-1],  compact=True)

[('parebens', 1), ('retificar', 1), ('totalizar', 1), ('contabilizando', 1),
 ('encurvados', 1), ('locar', 1), ('morava', 1), ('finda', 1), ('escovo', 1),
 ('pesquisávamos', 1), ('port', 1), ('traições', 1), ('dramas', 1),
 ('mastercad', 1), ('credores', 1), ('sugunda', 1), ('performa', 1),
 ('ahshsh', 1), ('caçadores', 1), ('descepicionado', 1), ('edn', 1), ('elm', 1),
 ('cente', 1), ('sgestão', 1), ('agor', 1), ('inrresponssabilidade', 1),
 ('flashes', 1), ('mamiss', 1), ('cumpádis', 1), ('casebre', 1),
 ('lampiões', 1), ('pavios', 1), ('geradorzinho', 1), ('gritava', 1),
 ('edtou', 1), ('silvestre', 1), ('lucio', 1), ('aparlho', 1), ('ofereces', 1),
 ('proprios', 1), ('coadores', 1), ('ncomprei', 1), ('saltadas', 1),
 ('capturadas', 1), ('empenamento', 1), ('durinha', 1), ('companheiros', 1),
 ('axhei', 1), ('reponde', 1), ('duais', 1), ('pasaram', 1), ('avançadinha', 1),
 ('dsrl', 1), ('explicaram', 1), ('fatando', 1), ('obirgado', 1), ('kiosk', 1),
 ('retornavão', 1), ('pwlo', 1),

In [14]:
short_words = [word for word in vocab if len(word) <= 3]

# Show some short words
pprint(short_words[:105], compact=True)

['não', 'tem', 'um', 'tv', 'box', 'dia', 'e', 'ate', 'as', 'o', 'meu', 'ja',
 'fiz', 'de', 'pre', 'em', 'com', 'mas', 'os', 'que', 'sao', 'por', 'ela',
 'mim', 'se', 'bom', 'seu', 'é', 'som', 'a', 'já', 'vi', 'vc', 'no', 'me',
 'foi', 'da', 'do', 'hp', 'sem', 'boa', 'ele', 'nem', 'bem', 'eu', 'só', 'voz',
 'na', 'qdo', 'vê', 'tô', 'ano', 'uso', 'j', 'ao', 'mão', 'era', 'eté', 'ser',
 'sou', 'até', 'deu', 'uma', 'há', 'ou', 'vou', 'b', 'aos', 'sai', 'dor', 'fiu',
 'ter', 'nen', 'são', 'ir', 'ex', 'vem', 'mal', 'sei', 'for', 'vcs', 'tim',
 'usa', 'g', 'abc', 'x', 'dvd', 'si', 'ok', 'pós', 'az', 'faz', 'vim', 's',
 'lg', 'sim', 'irá', 'vir', 'ves', 'nas', 'dá', 'tão', 'z', 'hd', 'vez']


## Training and evaluating some models

In [15]:
%%time

X = raw_train_data['review_clean']
y = raw_train_data['rating']

vectorizer = TfidfVectorizer(strip_accents='unicode', # remove accents                              
                             token_pattern=r'\w{2,}',
                             ngram_range=(1, 3),
                             min_df=3,
                             use_idf=1, 
                             smooth_idf=1, 
                             sublinear_tf=1)

models = [
    ('Naive Bayes', MultinomialNB()),
    ('Logistic Regression', LogisticRegression(class_weight='balanced', random_state=0, n_jobs=-1)),
    ('SVM', LinearSVC(class_weight='balanced', random_state=0)),
    ('KNN', KNeighborsClassifier(n_jobs=-1)),
    ('Decision Tree', DecisionTreeClassifier(random_state=0, class_weight='balanced')),    
    ('Random Forest', RandomForestClassifier(n_jobs=-1, random_state=0, class_weight='balanced')), 
    ('Extra Trees', ExtraTreesClassifier(n_jobs=-1, random_state=0, class_weight='balanced')), 
    ('XGBoost', XGBClassifier(random_state=0, n_jobs=-1)), 
    ('LightGBM', LGBMClassifier(objective='multiclass',class_weight='balanced', random_state=0, n_jobs=-1))
]

# Perform cross-validation
for name, model in tqdm(models, leave=None):

    pipeline = Pipeline([('vectorizer', vectorizer),
                         (name, model)])
    
    scores = cross_val_score(pipeline, X, y, scoring='accuracy')
    
    acc = np.mean(scores)
    std = np.std(scores)
    
    print(f'Accuracy {acc:.2%} +/- {std:.2%} = {(acc - std):.2%} -- {name}.')

  0%|          | 0/9 [00:00<?, ?it/s]

Accuracy 59.74% +/- 0.20% = 59.54% -- Naive Bayes.
Accuracy 62.61% +/- 0.36% = 62.24% -- Logistic Regression.
Accuracy 62.05% +/- 0.25% = 61.80% -- SVM.
Accuracy 28.65% +/- 0.95% = 27.70% -- KNN.
Accuracy 52.70% +/- 0.34% = 52.36% -- Decision Tree.
Accuracy 61.87% +/- 0.22% = 61.65% -- Random Forest.
Accuracy 61.45% +/- 0.24% = 61.21% -- Extra Trees.


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:53:59] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[22:05:04] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:10:36] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:16:09] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy 63.49% +/- 0.37% = 63.12% -- XGBoost.
Accuracy 61.19% +/- 0.20% = 60.99% -- LightGBM.
CPU times: user 3h 5min 41s, sys: 3min 51s, total: 3h 9min 32s
Wall time: 2h 44min 10s
